In [1]:
import pypsa 
import pandas as pd
import numpy as np

## Einlesen der Datenreihen

In [21]:
#infeed renewables
data_pv = pd.read_csv('data/data_pv_antofagasta_1MW.csv', skiprows = 3)['electricity']
data_wind = pd.read_csv('data/data_wind_antofagasta_1MW.csv', skiprows = 3)['electricity']
data_load = pd.read_csv('data/load_hydrogen.csv',header = 0, sep= ';', decimal = ',')['elektrische Last']
data_shipping_1 = pd.read_excel('data/Ferntransport.xlsx')['Kapazität [MWh]']
data_shipping_2 = 0

In [19]:
data_wind#.plot(figsize = (15,8))

0       33.207
1        6.290
2        0.437
3        0.198
4        0.164
         ...  
8779    26.414
8780    19.461
8781    11.750
8782     1.972
8783     0.000
Name: electricity, Length: 8784, dtype: float64

Multilink aufbauen

override_component_attrs = pypsa.descriptors.Dict(
    {k: v.copy() for k, v in pypsa.components.component_attrs.items()}
)
override_component_attrs["Link"].loc["bus2"] = [
    "string",
    np.nan,
    np.nan,
    "2nd bus",
    "Input (optional)",
]
override_component_attrs["Link"].loc["efficiency2"] = [
    "static or series",
    "per unit",
    1.0,
    "2nd bus efficiency",
    "Input (optional)",
]
override_component_attrs["Link"].loc["p2"] = [
    "series",
    "MW",
    0.0,
    "2nd bus output",
    "Output",
]

## Definition der Variablen

#### Investitionskosten

In [5]:
# generators
capital_cost_pv = 500 # €/MW
capital_cost_wind = 700 # €/MW

# links
capital_cost_PEMEL = 10 # €/MW
capital_cost_AEL = 12 # €/MW
capital_cost_HTEL = 13 # €/MW
capital_cost_NH3 = 14 # €/MW
capital_cost_LH2 = 15 # €/MW
capital_cost_LHOC = 16 # €/MW
capital_cost_pipeline = 50 # €/MW
capital_cost_shipping_1 = 100 # €/MW
capital_cost_shipping_2 = 80 # €/MW

# storages
capital_cost_storage_GH2 = 20 # €/MW
capital_cost_storage_NH3 = 20 # €/MW
capital_cost_storage_LH2 = 20 # €/MW
capital_cost_storage_LHOC = 20 # €/MW

#### Gestehungskosten

In [6]:
# generators
marginal_cost_pv = 50 # €/MWh
marginal_cost_wind = 70 # €/MWh

# links
marginal_cost_PEMEL = 0 # €/MWh
marginal_cost_AEL = 0 # €/MWh
marginal_cost_HTEL = 0 # €/MWh
marginal_cost_NH3 = 0 # €/MWh
marginal_cost_LH2 = 0 # €/MWh
marginal_cost_LHOC = 0 # €/MWh
marginal_cost_pipeline = 0 # €/MWh
marginal_cost_shipping_1 = 0 # €/MWh
marginal_cost_shipping_2 = 0 # €/MWh

# storage
marginal_cost_storage_GH2 = 0 # €/MWh
marginal_cost_storage_NH3 = 0 # €/MWh
marginal_cost_storage_LH2 = 0 # €/MWh
marginal_cost_storage_LHOC = 0 # €/MWh

# shipping
marginal_cost_shipping_GH2 = 0 # €/MWh
marginal_cost_shipping_NH3 = 0 # €/MWh
marginal_cost_shipping_LH2 = 0 # €/MWh
marginal_cost_shipping_LHOC = 0 # €/MWh

#### Sonstiges

In [7]:
# efficiencies
efficiency_PEMEL = 1
efficiency_AEL = 1
efficiency_HTEL = 1
efficiency_NH3 = 1
efficiency_LH2 = 1
efficiency_LHOC = 1

# lifetimes
lifetime_PEMEL = 20
lifetime_AEL = 20
lifetime_HTEL = 20

# standing losses
standing_loss_GH2 = 0.05
standing_loss_NH3 = 0.05
standing_loss_LH2 = 0.05
standing_loss_LHOC = 0.05

## Netzwerk aufbauen

In [8]:
network = pypsa.Network()
network.set_snapshots(range(8760))

#### Buses

In [9]:
network.add('Bus', name = 'electricity')
network.add('Bus', name = 'GH2')
#network.add('Bus', name = 'NH3')
#network.add('Bus', name = 'LH2')
#network.add('Bus', name = 'LHOC')
#network.add('Bus', name = 'hydrogen')

network.add('Bus', name = 'bremerhaven')
#network.add('Bus', name = 'cologne')

#### Loads

In [10]:
network.add('Load', name = 'hydrogen_demand', bus = 'bremerhaven', p_set = data_load)

#### Generators

In [11]:
network.add('Generator', name = 'pv', bus = 'electricity', 
            p_nom =  data_pv.max(), p_max_pu = data_pv/data_pv.max(), 
            marginal_cost = marginal_cost_pv, capital_cost = capital_cost_pv)

network.add('Generator', name = 'wind', bus = 'electricity', 
            p_nom =  data_wind.max(), p_max_pu = data_wind/data_wind.max(), 
            marginal_cost = marginal_cost_wind, capital_cost = capital_cost_wind)

#### Links - Electrolysis

In [12]:
network.add('Link', name = 'PEMEL', bus0 = 'electricity', bus1 = 'GH2',
           p_nom_extendable = True,
           marginal_cost = marginal_cost_PEMEL, capital_cost = capital_cost_PEMEL,
           efficiency = efficiency_PEMEL)

network.add('Link', name = 'AEL', bus0 = 'electricity', bus1 = 'GH2',
           p_nom_extendable = True,
           marginal_cost = marginal_cost_AEL, capital_cost = capital_cost_AEL,
           efficiency = efficiency_AEL)
network.add('Link', name = 'HTEL', bus0 = 'electricity', bus1 = 'GH2',
           p_nom_extendable = True,
           marginal_cost = marginal_cost_HTEL, capital_cost = capital_cost_HTEL,
           efficiency = efficiency_HTEL)

#### Links - Conversion

network.add('Link', name = 'NH3', bus0 = 'electricity', bus1 = 'NH3',
           p_nom_extendable = True,
           marginal_cost = marginal_cost_NH3, capital_cost = capital_cost_NH3,
           efficiency = efficiency_NH3)

network.add('Link', name = 'LH2', bus0 = 'electricity', bus1 = 'LH2',
           p_nom_extendable = True,
           marginal_cost = marginal_cost_LH2, capital_cost = capital_cost_LH2,
           efficiency = efficiency_LH2)

network.add('Link', name = 'LHOC', bus0 = 'electricity', bus1 = 'LHOC',
           p_nom_extendable = True,
           marginal_cost = marginal_cost_LHOC, capital_cost = capital_cost_LHOC,
           efficiency = efficiency_LHOC)

#### Links - Transportation

In [13]:
# long distance shipment
network.add('Link', name = 'Ferntransport_GH2', bus0 = 'GH2', bus1 = 'bremerhaven',
            p_nom = data_shipping_1.max(), p_max_pu = data_shipping_1/data_shipping_1.max(),
            marginal_cost = marginal_cost_shipping_GH2)

network.add('Link', name = 'Ferntransport_NH3', bus0 = 'NH3', bus1 = 'bremerhaven',
            p_nom = data_shipping_1.max(), p_max_pu = data_shipping_1/data_shipping_1.max(),
            marginal_cost = marginal_cost_shipping_NH3)

network.add('Link', name = 'Ferntransport_LH2', bus0 = 'LH2', bus1 = 'bremerhaven',
            p_nom = data_shipping_1.max(), p_max_pu = data_shipping_1/data_shipping_1.max(),
            marginal_cost = marginal_cost_shipping_LH2)

network.add('Link', name = 'Ferntransport_LHOC', bus0 = 'LHOC', bus1 = 'bremerhaven',
            p_nom = data_shipping_1.max(), p_max_pu = data_shipping_1/data_shipping_1.max(),
            marginal_cost = marginal_cost_shipping_LHOC)

# inland shipment
#TODO: Profil hinterlegen und Links einbauen

#### Storages

In [14]:
network.add('Store', name = 'hydrogen_store_1', bus = 'GH2', 
            e_nom_extendable = True,  e_cyclic = True, 
            capital_cost = capital_cost_storage_GH2)

network.add('Store', name = 'hydrogen_store_1.1', bus = 'NH3', 
            e_nom_extendable = True,  e_cyclic = True, 
            capital_cost = capital_cost_storage_NH3)

network.add('Store', name = 'hydrogen_store_1.2', bus = 'LH2', 
            e_nom_extendable = True, e_cyclic = True,  
            capital_cost = capital_cost_storage_LH2)

network.add('Store', name = 'hydrogen_store_1.3', bus = 'LHOC', 
            e_nom_extendable = True, e_cyclic = True,  
            capital_cost = capital_cost_storage_LHOC)

network.add('Store', name = 'hydrogen_store_2', bus = 'bremerhaven', 
            e_nom_extendable = True, e_cyclic = True, 
            capital_cost = capital_cost_storage_GH2)

network.add('Store', name = 'hydrogen_store_3', bus = 'cologne', 
            e_nom_extendable = True, e_cyclic = True, 
            capital_cost = capital_cost_storage_GH2)

## Optimierung des Systems

In [15]:
network.lopf(pyomo = False, solver_name = 'gurobi')

INFO:pypsa.linopf:Prepare linear problem
INFO:pypsa.linopf:Total preparation time: 1.14s
INFO:pypsa.linopf:Solve linear problem using Gurobi solver


Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-01
Error reading LP format file /var/folders/np/fxyv4bts2s353fj5gdlrbgym0000gn/T/pypsa-problem-rw489dp_.lp at line 227778
Unrecognized constraint RHS or sense
Neighboring tokens: " <= +nan c52561: +1.000000 x1 -1.000000 x26288 "

Unable to read file


GurobiError: Unable to read model

## Auswertung der Ergebnisse

In [16]:
network.generators_t.p.plot(figsize = (15,8))

TypeError: no numeric data to plot

In [ ]:
network.links_t.p0.plot(figsize = (15,8))

In [ ]:
network.stores_t.e.plot(figsize = (15,8))